In [1016]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import stock_module as sm
import random
import os
import datetime
import time
import openpyxl
from sklearn.preprocessing import scale
from scipy.signal import savgol_filter

In [1073]:
frame_ = pd.read_pickle("frame_whole_BIST50")

In [1074]:
def get_stock_frame(stock_name, start_date, end_date):
    frame_ = pd.read_pickle("frame_whole_BIST50")
    frame = frame_.loc[frame_["stock_name"] == stock_name]
    frame = frame.loc[frame.index > start_date]
    frame = frame.loc[frame.index < end_date]
    return frame

In [1075]:
def remove_noise(frame, first_par, second_par):
    w_out_noise = savgol_filter(frame["StockValue"], first_par, second_par)
    frame["StockValue"] = w_out_noise
    frame_f = frame[["StockValue"]]
    return frame_f

In [1076]:
def ma_columns(frame_column, list_ma):
    MAs = [] 
    for ma in list_ma:
        MA = pd.DataFrame(sm.short_moving_average(frame_column, ma)["StockValue"]).rename(columns={"StockValue": f"MA_{ma}"})
        MAs.append(MA)
        
    frame = MAs[0].join(MAs[1])
    for i in range(len(MAs)-2):
        frame = frame.join(MAs[i+2])
    
    frame = frame.dropna()
    
    return frame

In [1077]:
def buy_sell_recommendations(new_frame):
    new_frame['rec_1'] = np.where((new_frame['MA_7_50'] > new_frame["MA_50_200"])
                                         , "BUY", "SELL")
    new_frame['rec_2'] = np.where((new_frame['MA_14_100'] > new_frame["MA_50_200"])
                                         , "BUY", "SELL")
    return new_frame

In [1078]:
def get_locations(new_frame):
    loc_first=[]
    for row in range(new_frame.shape[0]-1):

            if new_frame.rec_1[row] == "SELL":
                if new_frame.rec_1[row+1] == "BUY":
                    start = row+1
                    loc_first.append(start)

            if new_frame.rec_1[row] == "BUY":
                if new_frame.rec_1[row+1] == "SELL":
                    end = row+2
                    loc_first.append(end)
                    
    loc_second=[]
    for row in range(new_frame.shape[0]-1):

        if new_frame.rec_2[row] == "SELL":
            if new_frame.rec_2[row+1] == "BUY":
                start = row+1
                loc_second.append(start)

        if new_frame.rec_2[row] == "BUY":
            if new_frame.rec_2[row+1] == "SELL":
                end = row+2
                loc_second.append(end)
                
    return [loc_first, loc_second]

In [1079]:
def get_transactions(new_frame, column, locations):
    if new_frame.iloc[0,:][str(column)] == "SELL":
        transactions = [[locations[buy], locations[sell]] for buy, sell in zip(range(0,len(locations),2), range(1,len(locations),2))]
    else:
        transactions = [[locations[buy], locations[sell]] for buy, sell in zip(range(1,len(locations),2), range(2,len(locations),2))]
        transactions.insert(0, [0, locations[0]])

    return transactions

In [1080]:
def returns(new_frame, list_of_transactions):
    returns = []
    for transactions in list_of_transactions:
        buy = new_frame.iloc[transactions[0]:transactions[1],:]["StockValue"][0]
        sell = new_frame.iloc[transactions[0]:transactions[1],:]["StockValue"][-1]
        ret = (sell-buy)/buy
        returns.append(ret)
    
    return returns

In [1081]:
def prod(list_of_all_returns):
    return np.prod([ret+1 for ret in list_of_all_returns]) 

In [1112]:
def simulation(stock_name, start_date, end_date, ma_list, noise_first_par, noise_second_par):
    frame = get_stock_frame(stock_name, start_date, end_date)
    frame_f = remove_noise(frame, noise_first_par, noise_second_par)
    new_frame = ma_columns(frame_f, ma_list)
    
    new_frame["MA_7_50"] = new_frame["MA_7"] - new_frame["MA_50"]
    new_frame["MA_14_100"] = new_frame["MA_14"] - new_frame["MA_100"]
    new_frame["MA_50_200"] = new_frame["MA_50"] - new_frame["MA_200"]
    
    frame = get_stock_frame(stock_name, start_date, end_date)
    frame = frame[["StockValue"]]
    new_frame = new_frame.join(frame)
    
    new_frame = buy_sell_recommendations(new_frame)
    
    loc_first = get_locations(new_frame)[0]
    loc_second = get_locations(new_frame)[1]
    
    first_transactions = get_transactions(new_frame, "rec_1", loc_first)
    second_transactions = get_transactions(new_frame, "rec_2", loc_second)

    first_returns = returns(new_frame, first_transactions)
    second_returns = returns(new_frame, second_transactions)
    
    first_cum_return = prod(first_returns)
    # print(first_cum_return)
    # print(sum([t[1]-t[0] for t in first_transactions]))
    
    second_cum_return  = prod(second_returns)
    # print(second_cum_return)
    # print(sum([t[1]-t[0] for t in second_transactions]))

    totalReturn = (first_cum_return + second_cum_return) / 2
    
    return totalReturn

In [1139]:
simulation("TKFEN", "01-01-2015", "07-07-2020", [7,14,50,100,200], 51, 2)

2.8248085240860448

In [1140]:
results = []
for first_param in range(11, 201, 20):
    results.append(simulation("TKFEN", "01-01-2015", "07-07-2020", [7,14,50,100,200], first_param, 2))
    

In [1141]:
results

[2.243232832994107,
 2.4975990869700433,
 2.8248085240860448,
 2.8946276950787926,
 2.3029934146981184,
 2.3991107516057735,
 2.6329307365973866,
 2.3650920123664534,
 2.323405618658817,
 2.3808750568472217]

In [1142]:
TKFEN = results

In [1143]:
sm.volatility("TKFEN", start_date, end_date)  

0.02294511207425897

In [1144]:
TKFEN

[2.243232832994107,
 2.4975990869700433,
 2.8248085240860448,
 2.8946276950787926,
 2.3029934146981184,
 2.3991107516057735,
 2.6329307365973866,
 2.3650920123664534,
 2.323405618658817,
 2.3808750568472217]

In [1146]:
[i for i in range(11, 201, 20)]

[11, 31, 51, 71, 91, 111, 131, 151, 171, 191]

---

In [1135]:
SASA = results

In [1134]:
sm.volatility("SASA", start_date, end_date)  

0.030740089200642154

In [1136]:
SASA

[5.395995361018599,
 9.361991029979574,
 11.889918443580596,
 11.620152448759342,
 12.5831659383134,
 12.725278649568313,
 12.038704299950211,
 7.588809698334817,
 6.575260153421684,
 3.9127747894269786,
 5.162111462399878,
 3.2782732713046623,
 3.443490108578751,
 2.7864607695351586,
 2.6262581338294]

In [1137]:
[i for i in range(11, 301, 20)]

[11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 231, 251, 271, 291]

---

In [1131]:
sm.volatility("IPEKE", start_date, end_date)  

0.04032749848428386

In [1127]:
IPEKE

[6.506137653444306,
 10.935850030993866,
 12.807757666463791,
 13.327239672874232,
 12.236232419189536,
 11.669831183217106,
 11.641439225773295,
 10.55952448584773,
 7.221713909522094,
 7.110923152420741,
 6.151811144617959,
 6.197281181687792,
 6.553234840278719,
 3.1130061525774337,
 2.5619245790258454]

In [1128]:
[i for i in range(11, 301, 20)]

[11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 231, 251, 271, 291]

---

In [1129]:
sm.volatility("BIMAS", start_date, end_date)  

0.01564850164225352

In [1120]:
[i for i in range(11, 301, 20)]

[11, 31, 51, 71, 91, 111, 131, 151, 171, 191, 211, 231, 251, 271, 291]

In [1123]:
BIMAS

[1.8836155601882512,
 2.0797884381209055,
 2.2631752537491154,
 2.3056161798194585,
 2.447694322049661,
 2.45792139495264,
 2.5165630318769985,
 2.171200579029925,
 2.1762835699926204,
 1.9823620363293661,
 1.7817437265937972,
 2.0491894364948893,
 1.63607027483424,
 1.4445318639096643,
 1.4477884785744544]

In [1105]:
start_date = "01-01-2015"
end_date = "01-01-2020"

In [1107]:
sm.volatility("BIMAS", start_date, end_date)  

0.01564850164225352

In [ ]:
w_out_noise = savgol_filter(frame["StockValue"], 51, 2)

In [1018]:
start_date = "01-03-2015"
end_date = "11-01-2021"
stock_name = "IPEKE"

In [1019]:
frame = frame_.loc[frame_["stock_name"] == stock_name]
frame = frame.loc[frame.index > start_date]
frame = frame.loc[frame.index < end_date]

In [1020]:
frame

,MA_7,MA_14,MA_20,MA_50,MA_100,MA_200,MA_7_CH,MA_7_CH_CH,MA_14_CH,MA_14_CH_CH,...,RSI_5_CH,Drawdown,marketV_to_bookV,StockValue,StockValue_momentum,market_positive,market_positive_ma_10,stock_name,volume,volume_ma_14
2015-01-05,1.575714,1.571429,1.5660,1.6274,1.8116,2.21795,0.000676,-0.566415,-0.001089,-0.039963,...,NaN,-0.744337,0.163191,1.58,108.219178,0.524590,0.539108,IPEKE,6762649.0,6.744830e+06
2015-01-06,1.580000,1.583571,1.5670,1.6240,1.8042,2.21280,0.001916,-0.445477,0.000166,-0.146177,...,NaN,-0.737864,0.167323,1.62,111.724138,0.565217,0.517369,IPEKE,9360091.0,7.111015e+06
2015-01-07,1.590000,1.591429,1.5705,1.6214,1.7973,2.20785,0.002633,-0.418982,0.000843,0.136407,...,0.150538,-0.729773,0.172487,1.67,107.051282,0.451505,0.514436,IPEKE,14441579.0,6.272524e+06
2015-01-08,1.601429,1.596429,1.5740,1.6188,1.7905,2.20265,0.003346,-0.397366,0.001230,0.162954,...,0.120984,-0.734628,0.169388,1.64,104.458599,0.656863,0.519105,IPEKE,3872396.0,6.153627e+06
2015-01-09,1.611429,1.601429,1.5780,1.6164,1.7836,2.19760,0.003859,-0.382007,0.001649,0.177003,...,0.084893,-0.736246,0.168355,1.63,104.487179,0.430464,0.503957,IPEKE,2000164.0,5.547239e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-15,14.261429,14.511429,14.3380,12.2740,10.5571,8.71450,0.001635,-0.124410,0.004549,-0.084776,...,-0.012043,-0.083333,0.567216,14.19,100.282686,0.481579,0.512819,IPEKE,330639890.0,2.180499e+08
2020-06-16,14.192857,14.492143,14.3345,12.3954,10.6098,8.75445,0.001050,-0.141492,0.003716,-0.095580,...,-0.013033,-0.090439,0.567216,14.08,98.118467,0.829396,0.539963,IPEKE,89042541.0,2.117746e+08
2020-06-17,14.180000,14.385000,14.3385,12.5204,10.6492,8.79445,0.000015,-0.203632,0.002376,-0.117612,...,-0.020861,-0.096899,0.567216,13.98,90.310078,0.424403,0.513070,IPEKE,80202706.0,1.812783e+08
2020-06-18,14.088571,14.333571,14.3645,12.6442,10.6933,8.83340,-0.001057,-5.241481,0.002007,-0.120681,...,-0.023787,-0.093669,0.567216,14.03,95.118644,0.583554,0.533244,IPEKE,173388556.0,1.746990e+08


In [1021]:
w_out_noise = savgol_filter(frame["StockValue"], 51, 2)

In [1022]:
frame["StockValue"] = w_out_noise

In [1023]:
frame_f = frame[["StockValue"]]

In [1024]:
def ma_columns(frame_column, list_ma):
    MAs = [] 
    for ma in list_ma:
        MA = pd.DataFrame(sm.short_moving_average(frame_column, ma)["StockValue"]).rename(columns={"StockValue": f"MA_{ma}"})
        MAs.append(MA)
        
    frame = MAs[0].join(MAs[1])
    for i in range(len(MAs)-2):
        frame = frame.join(MAs[i+2])
    
    frame = frame.dropna()
    
    return frame

In [1025]:
new_frame = ma_columns(frame_f, [7,14,50,100,200])

In [1026]:
new_frame["MA_7_50"] = new_frame["MA_7"] - new_frame["MA_50"]
new_frame["MA_14_100"] = new_frame["MA_14"] - new_frame["MA_100"]
new_frame["MA_50_200"] = new_frame["MA_50"] - new_frame["MA_200"]

In [1027]:
new_frame

,MA_7,MA_14,MA_50,MA_100,MA_200,MA_7_50,MA_14_100,MA_50_200
2015-10-19,1.927026,1.925131,2.008585,2.388976,2.269154,-0.081559,-0.463846,-0.260568
2015-10-20,1.917829,1.923858,1.997654,2.379197,2.270889,-0.079825,-0.455339,-0.273235
2015-10-21,1.906346,1.920616,1.986841,2.369351,2.272491,-0.080495,-0.448735,-0.285650
2015-10-22,1.892166,1.915087,1.976119,2.359415,2.273947,-0.083952,-0.444328,-0.297829
2015-10-23,1.875268,1.907309,1.965464,2.349306,2.275248,-0.090197,-0.441997,-0.309784
...,...,...,...,...,...,...,...,...
2020-06-15,14.393332,14.399496,12.202136,10.537121,8.715884,2.191196,3.862375,3.486252
2020-06-16,14.357420,14.393466,12.335768,10.585220,8.756810,2.021652,3.808246,3.578958
2020-06-17,14.312971,14.378898,12.464662,10.632297,8.797230,1.848309,3.746601,3.667432
2020-06-18,14.259984,14.355793,12.587796,10.678141,8.837100,1.672187,3.677652,3.750696


In [1028]:
# new_frame["MA_50_200"] = scale(new_frame["MA_50_200"])
# new_frame["MA_14_100"] = scale(new_frame["MA_14_100"])
# new_frame["MA_7_50"] = scale(new_frame["MA_7_50"])

In [1029]:
frame = frame_.loc[frame_["stock_name"] == stock_name]
frame = frame.loc[frame.index > start_date]
frame = frame.loc[frame.index < end_date]

In [1030]:
frame = frame[["StockValue"]]

In [1031]:
new_frame = new_frame.join(frame)

In [1057]:
def buy_sell_recommendations(new_frame):
    new_frame['rec_1'] = np.where((new_frame['MA_7_50'] > new_frame["MA_50_200"])
                                         , "BUY", "SELL")
    new_frame['rec_2'] = np.where((new_frame['MA_14_100'] > new_frame["MA_50_200"])
                                         , "BUY", "SELL")
    return new_frame

In [1032]:
new_frame['rec_1'] = np.where((new_frame['MA_7_50'] > new_frame["MA_50_200"])
                                         , "BUY", "SELL")

In [1033]:
new_frame['rec_2'] = np.where((new_frame['MA_14_100'] > new_frame["MA_50_200"])
                                         , "BUY", "SELL")

In [1061]:
def get_locations(new_frame):
    loc_first=[]
    for row in range(new_frame.shape[0]-1):

            if new_frame.rec_1[row] == "SELL":
                if new_frame.rec_1[row+1] == "BUY":
                    start = row+1
                    loc_first.append(start)

            if new_frame.rec_1[row] == "BUY":
                if new_frame.rec_1[row+1] == "SELL":
                    end = row+2
                    loc_first.append(end)
                    
    loc_second=[]
    for row in range(new_frame.shape[0]-1):

        if new_frame.rec_2[row] == "SELL":
            if new_frame.rec_2[row+1] == "BUY":
                start = row+1
                loc_second.append(start)

        if new_frame.rec_2[row] == "BUY":
            if new_frame.rec_2[row+1] == "SELL":
                end = row+2
                loc_second.append(end)
                
    return [loc_first, loc_second]

In [1064]:
get_locations(new_frame)[1]

[42,
 172,
 218,
 278,
 414,
 499,
 628,
 638,
 653,
 691,
 772,
 836,
 903,
 1003,
 1058,
 1092,
 1138,
 1173]

In [1034]:
loc_first=[]
for row in range(new_frame.shape[0]-1):

        if new_frame.rec_1[row] == "SELL":
            if new_frame.rec_1[row+1] == "BUY":
                start = row+1
                loc_first.append(start)

        if new_frame.rec_1[row] == "BUY":
            if new_frame.rec_1[row+1] == "SELL":
                end = row+2
                loc_first.append(end)

In [1035]:
loc_second=[]
for row in range(new_frame.shape[0]-1):

    if new_frame.rec_2[row] == "SELL":
        if new_frame.rec_2[row+1] == "BUY":
            start = row+1
            loc_second.append(start)
            
    if new_frame.rec_2[row] == "BUY":
        if new_frame.rec_2[row+1] == "SELL":
            end = row+2
            loc_second.append(end)

In [1036]:
loc_first

[159,
 202,
 265,
 408,
 472,
 626,
 637,
 648,
 685,
 734,
 814,
 880,
 992,
 1059,
 1075,
 1134,
 1165]

In [1037]:
loc_second

[42,
 172,
 218,
 278,
 414,
 499,
 628,
 638,
 653,
 691,
 772,
 836,
 903,
 1003,
 1058,
 1092,
 1138,
 1173]

In [1038]:
def get_transactions(new_frame, column, locations):
    if new_frame.iloc[0,:][str(column)] == "SELL":
        transactions = [[locations[buy], locations[sell]] for buy, sell in zip(range(0,len(locations),2), range(1,len(locations),2))]
    else:
        transactions = [[locations[buy], locations[sell]] for buy, sell in zip(range(1,len(locations),2), range(2,len(locations),2))]
        transactions.insert(0, [0, locations[0]])

    return transactions

In [1039]:
first_transactions = get_transactions(new_frame, "rec_1", loc_first)
first_transactions

[[0, 159],
 [202, 265],
 [408, 472],
 [626, 637],
 [648, 685],
 [734, 814],
 [880, 992],
 [1059, 1075],
 [1134, 1165]]

In [1040]:
second_transactions = get_transactions(new_frame, "rec_2", loc_second)
second_transactions

[[42, 172],
 [218, 278],
 [414, 499],
 [628, 638],
 [653, 691],
 [772, 836],
 [903, 1003],
 [1058, 1092],
 [1138, 1173]]

In [1041]:
def returns(new_frame, list_of_transactions):
    returns = []
    for transactions in list_of_transactions:
        buy = new_frame.iloc[transactions[0]:transactions[1],:]["StockValue"][0]
        sell = new_frame.iloc[transactions[0]:transactions[1],:]["StockValue"][-1]
        ret = (sell-buy)/buy
        returns.append(ret)
    
    return returns

In [1042]:
first_transactions

[[0, 159],
 [202, 265],
 [408, 472],
 [626, 637],
 [648, 685],
 [734, 814],
 [880, 992],
 [1059, 1075],
 [1134, 1165]]

In [1043]:
first_returns = returns(new_frame, first_transactions)
first_returns

[-0.38829787234042556,
 1.6874999999999998,
 1.3846153846153848,
 -0.01744186046511625,
 0.23809523809523805,
 0.17634854771784225,
 0.4901531728665206,
 0.0599781897491822,
 0.3515850144092219]

In [1044]:
second_transactions

[[42, 172],
 [218, 278],
 [414, 499],
 [628, 638],
 [653, 691],
 [772, 836],
 [903, 1003],
 [1058, 1092],
 [1138, 1173]]

In [1045]:
second_returns = returns(new_frame, second_transactions)
second_returns

[0.19277108433734944,
 0.8230088495575223,
 2.675531914893617,
 0.012269938650306851,
 0.0,
 0.07916666666666665,
 0.2520325203252033,
 0.09448818897637794,
 0.14530612244897953]

In [1046]:
new_frame.shape[0]

1174

In [1047]:
sum([t[1]-t[0] for t in first_transactions])

573

In [1048]:
sum([t[1]-t[0] for t in second_transactions])

556

In [1049]:
def prod(list_of_all_returns):
    return np.prod([ret+1 for ret in list_of_all_returns]) 

In [1050]:
first_cum_return = prod(first_returns)
first_cum_return

11.976401477421598

In [1051]:
second_cum_return  = prod(second_returns)
second_cum_return

13.702481059090225

In [1052]:
totalReturn = (first_cum_return + second_cum_return) / 2
totalReturn

12.83944126825591

In [1053]:
new_frame.iloc[-20:,:]

,MA_7,MA_14,MA_50,MA_100,MA_200,MA_7_50,MA_14_100,MA_50_200,StockValue,rec_1,rec_2
2020-05-21,13.907487,13.422845,10.125435,9.691326,8.066259,3.782052,3.731519,2.059177,14.50,BUY,BUY
2020-05-22,13.999640,13.558054,10.253294,9.753248,8.110563,3.746346,3.804806,2.142731,14.15,BUY,BUY
2020-05-27,14.083256,13.682636,10.384367,9.814772,8.154894,3.698889,3.867864,2.229474,14.35,BUY,BUY
2020-05-28,14.158334,13.795030,10.518374,9.875780,8.199185,3.639960,3.919250,2.319189,15.48,BUY,BUY
2020-05-29,14.224875,13.896981,10.654811,9.936198,8.243376,3.570064,3.960784,2.411435,14.75,BUY,BUY
2020-06-01,14.282877,13.988487,10.793172,9.995841,8.287417,3.489705,3.992646,2.505755,14.50,BUY,BUY
2020-06-02,14.332342,14.069181,10.933105,10.054698,8.331295,3.399238,4.014482,2.601809,15.02,BUY,BUY
2020-06-03,14.373270,14.140378,11.074169,10.112628,8.375016,3.299100,4.027751,2.699153,14.63,BUY,BUY
2020-06-04,14.405659,14.202650,11.216181,10.169625,8.418537,3.189478,4.033025,2.797644,14.60,BUY,BUY
2020-06-05,14.429511,14.256384,11.359055,10.225594,8.461881,3.070457,4.030790,2.897174,14.56,BUY,BUY
